<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Logo_ITESO_normal.jpg/240px-Logo_ITESO_normal.jpg" width="90px" height="35px" />


# Laboratorio 1 parte 2 equipo 2
## <font color= #8A0829> Microestructura y Sistemas de Trading.</font>
- <Strong> Jazmin Calderon Erape </Strong>

___
### <Strong> <font color = skyblue> I. Motivación del Laboratorio </Strong>

> <p style='text-align: justify;'>Acabas de entrar a trabajar a una operadora de fondos de inversión como analista financiero dentro de la mesa de análisis del mercado de renta variable. En la junta trimestral se plantea la posibilidad de crear un fondo propio con exposición al mercado de valores de USA tratando de replicar a los ETFs conocidos como ishares que la empresa BlackRock ofrece.

> <p style='text-align: justify;'>La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.

___
### <Strong> <font color = skyblue> II. Objetivos específicos </Strong>

* Construir un código para cualquier ETF con información de 1 año: 1.3.17al 1.3.18. Con el objetivo de conocer la rentabilidad y cada cuanto hay rebalanceo.
* Conocer las comisiones que se incurre por rebalanceo
* Calcular 3 medidas de atribución al desempeño, tanto para el caso que el ETF nunca hubiese sido rebalanceado como para los casos donde sí.

##### U.S. Technology ETF
Para este trabajo se eligio el ETF U.S. Technology de la plataforma de iShares el cual busca realizar un seguimiento de los resultados de inversión de un índice compuesto por acciones de Estados Unidos en el sector tecnológico.

* Base de datos - De dicho ETF se selecciono un año (Marzo 17 - Marzo 18). Se recopilaron 12 archivos mensuales en total.


In [17]:
#Laboratorio 1 parte 2 equipo 2

# Remover todos los objetos del "Environment"
rm(list = ls())

# los 0s aceptados antes de expresas una cifra en notaci?n cient?fica
options("scipen"=100, "digits"=4)

###############################################################################################
################### Cargar librerias a utilizar ###############################################
suppressMessages(library(plotly)) # Graficas interactivas
suppressMessages(library(Quandl)) # Descargar Precios
suppressMessages(library(PortfolioAnalytics)) # Teor?a Moderna de Portafolios
suppressMessages(library(ROI)) # Optimizacion para portafolio
suppressMessages(library(knitr))  # Opciones de documentaci?n + c?digo
suppressMessages(library(kableExtra)) # Tablas en HTML
options(knitr.table.format = "html") 


Warning message:
"package 'plotly' was built under R version 3.5.2"Warning message:
"package 'ggplot2' was built under R version 3.5.2"Warning message:
"package 'Quandl' was built under R version 3.5.2"Warning message:
"package 'xts' was built under R version 3.5.2"Warning message:
"package 'zoo' was built under R version 3.5.2"Warning message:
"package 'PortfolioAnalytics' was built under R version 3.5.2"Warning message:
"package 'foreach' was built under R version 3.5.2"Warning message:
"package 'PerformanceAnalytics' was built under R version 3.5.2"Warning message:
"package 'ROI' was built under R version 3.5.2"Warning message:
"package 'knitr' was built under R version 3.5.2"Warning message:
"package 'kableExtra' was built under R version 3.5.2"

In [18]:
###############################################################################################
################# Función para leer los 12 archivos ###########################################

funcion_pesos <- function(i){
  
  #i <- 1
  f <- paste("~/Trading/lab1/IYW_holdings_",i,".csv", sep="")
  datos_iak <- read.delim2(f)
  primero <- which(x = datos_iak[,1] == "Ticker")
  ultimo <- length(datos_iak[,1])
  
  tk <- datos_iak[(primero+1):ultimo,1]
  pesos <- datos_iak[(primero+1):ultimo,4]
  #pesos1 <- as.numeric(as.character(pesos))
  fecha <- datos_iak[2,2]
  titulos_ETF <- datos_iak[4,2]
  precios <- datos_iak[(primero+1):ultimo,5]
  #precios1 <- as.numeric(as.character(precios))
  shares <- datos_iak[(primero+1):ultimo,6]
  mv <- datos_iak[(primero+1):ultimo,7]
  
  
  resultado <- data.frame(tk,pesos,precios,shares,mv, titulos_ETF) 
  
  return(resultado)
  
}
#uso de la función
base_datos <- list()

for (i in 1:12){
  base_datos[[i]] <- funcion_pesos(i)
} 

In [19]:
base_datos

tk,pesos,precios,shares,mv,titulos_ETF
AAPL,17.71,143.66,"4,038,502.00","580,171,197.32","24,300,000.00"
MSFT,11.96,65.86,"5,948,236.00","391,750,822.96","24,300,000.00"
FB,7.86,142.05,"1,812,881.00","257,519,746.05","24,300,000.00"
GOOGL,5.92,847.8,"228,704.00","193,895,251.20","24,300,000.00"
GOOG,5.76,829.56,"227,459.00","188,690,888.04","24,300,000.00"
INTC,4.01,36.07,"3,639,358.00","131,271,643.06","24,300,000.00"
CSCO,3.98,33.8,"3,854,777.00","130,291,462.60","24,300,000.00"
IBM,3.51,174.14,"660,692.00","115,052,904.88","24,300,000.00"
ORCL,3.14,44.61,"2,306,977.00","102,914,243.97","24,300,000.00"
AVGO,2.06,218.96,"308,700.00","67,592,952.00","24,300,000.00"


In [20]:
# Cargar el token de QUANDL
Quandl.api_key("1YRXno4c2f3x5e96L3Dx")
Capital_Inicial <- 999000000
Comision <- 0.0003 #% de lo que se compra o vende
# Funcion para descagar precios
Bajar_Precios <- function(Columns, Tickers, Fecha_In, Fecha_Fn) {
  
  # Funcion para descargar N cantidad de activos desde QUANDL
  # -- Dependencias: QUANDL
  # -- Columns : columnas a incluir : character : c("date", "adj_close", ... )
  # -- Tickers : Tickers o claves de pizarra de los activos : character : "TSLA"
  # -- Fecha_In : Fecha Inicial : character : "2017-01-02"
  # -- Fecha_Fn : Fecha Final : character : "2017-08-02"
  
  # Peticion para descargar precios
  Datos <- Quandl.datatable("WIKI/PRICES", qopts.columns=Columns, ticker=Tickers,
                            date.gte=Fecha_In, date.lte=Fecha_Fn)
  
  
  return(Datos)
}


In [21]:
#Función compra venta para conocer que acciones se compraron o vendieron de un mes a otro
venta_compra <- function(a, b){
  
  # estos los tengo que comprar al precio de B
  sequitaron <- a[!(a %in% b)]
  
  # estos son los que tengo que vender
  seincluyeron <- b[!(b %in% a)]
  
  # datos que regresa la funcion
  resultado <- list("sequitaron"=sequitaron, "seincluyeron"=seincluyeron)
  
  return(resultado)
}


Como primer paso se calculara el Net Asset Value de cada mes para posteriormente calcular los rendimientos de cada serie. Entotal seran 12 series:

* Serie 1 - Marzo 17 - Marzo 18
* Serie 2 - Abril 17 - Marzo 18
* Serie 3 - Mayo 17 - Marzo 18
* Serie 4 - Junio 17 - Marzo 18
* Serie 5 - Julio 17 - Marzo 18
* Serie 6 - Agosto - Marzo 18
*  Serie 7 - Septiembre 17 - Marzo 18
* Serie 8 - Octubre 17 - Marzo 18
* Serie 9 - Novimbre 17 - Marzo 18
* Serie 10 - Diciembre 17 - Marzo 18
* Serie 11 - Enero 18 - Marzo 18
* Serie 12 - Febrero 18 - Marzo 18

Formulas que se utilizaron:
Net Asset Value
$$NAV=\frac {\sum Market Value}{No. Titulos ETF}$$
Rendimiento
$$Rend=\frac {x_1}{x_-1}-1$$

In [22]:
######### NAV para rendimiento SERIE 1 ###############
replaceCommas<-function(x){ #Funcion para quitar comas de las cantidades ya que en R no los toma como num si tienen dichas comas
  x<-as.numeric(gsub('[^a-zA-Z0-9.]', '', x))
}

NAV_serie1 <- (base_datos[[1]][['mv']]) #lectura de la columna market value
NAV_serie1<- NAV_serie1[-139] 
NAV_serie1<-replaceCommas(NAV_serie1)
NAV_serie1<- sum(NAV_serie1)/24300000

######### NAV para rendimiento SERIE 12  (Feb18) ###############
NAV_serie12 <- (base_datos[[12]][['mv']])
NAV_serie12<- NAV_serie12[-143] 
NAV_serie12<-replaceCommas(NAV_serie12)
NAV_serie12<- sum(NAV_serie12)/24150000

In [23]:
NAV_serie1


[1] 134.8

In [24]:
NAV_serie12

[1] 175.1

In [27]:
######### Rendimiento serie original (Mar17-Feb18)###############

A1 <- as.character(base_datos[[1]][,1]) #abril 17
B1 <- as.character(base_datos[[12]][,1]) #febrero 18

reg1 <- venta_compra(a=A1, b=B1)
shares_venta <- (base_datos[[1]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 1 ######
for(i in 1:length(reg1$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[1]]$tk == reg1$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta_1<-c()
    #print(k)
    titulos_venta1 <- (base_datos[[1]][['shares']])
    titulos_venta1<- titulos_venta1[-139] 
    titulos_venta1<-replaceCommas(titulos_venta1)
    titulos_venta_1[k]<- (titulos_venta1[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[1]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi1<-sum(dinero*titulos_venta_1)
    
  }
}

In [28]:
vendi1

[1] 61364720

In [31]:
##### CACULAR EL GASTO POR compra DE TK SERIE 1 ######
for(j in 1:length(reg1$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg1$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra_1<-c()
    #print(p)
    titulos_compra1 <- (base_datos[[12]][['shares']])
    titulos_compra1<- titulos_compra1[-143] 
    titulos_compra1<-replaceCommas(titulos_compra1)
    titulos_compra_1[p]<- (titulos_compra1[compra[p]])
   # dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre1<-sum(dinero*titulos_compra_1)
    #gasto_compra1<- compre*Comision
    
  }
}

In [32]:
compre1

[1] 75833526

In [33]:
NAV_serie121 <- (base_datos[[12]][['mv']])
NAV_serie121<- NAV_serie121[-143] 
NAV_serie121<-replaceCommas(NAV_serie121)
NAV_serie121<- (sum(NAV_serie121)+vendi1)-compre1
NAV_serie121<- NAV_serie121/24300000
rend1<-(NAV_serie121/NAV_serie1)-1

In [34]:
rend1

[1] 0.2863

In [36]:

######### SERIE 2 (Abril 17) ###############
A2 <- as.character(base_datos[[2]][,1]) #abril 17
B2 <- as.character(base_datos[[12]][,1]) #febrero 18

reg2 <- venta_compra(a=A2, b=B2)
shares_venta <- (base_datos[[2]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 2######
for(i in 1:length(reg2$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[2]]$tk == reg2$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta2 <- (base_datos[[2]][['shares']])
    titulos_venta2<- titulos_venta2[-143] 
    titulos_venta2<-replaceCommas(titulos_venta2)
    titulos_venta[k]<- (titulos_venta2[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[2]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta2<- vendi*Comision

  }
}

In [37]:
capital_dventa2<- (Capital_Inicial+ vendi)-gasto_venta2

In [38]:
gasto_venta2

[1] 16714

In [40]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 2######
for(j in 1:length(reg2$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg2$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra2 <- (base_datos[[12]][['shares']])
    titulos_compra2<- titulos_compra2[-143] 
    titulos_compra2<-replaceCommas(titulos_compra2)
    titulos_compra[p]<- (titulos_compra2[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra2<- compre*Comision
    
  }
}
capital_dcompra2<- (capital_dventa2 - compre)-gasto_compra2 #capital despues de comisiones por compra-venta


In [41]:
gasto_compra2

[1] 15997

In [42]:
######### NAV para rendimiento SERIE 2 (Abril 17)  ###############
NAV_serie2 <- (base_datos[[2]][['mv']])
NAV_serie2<- NAV_serie2[-139] 
NAV_serie2<-replaceCommas(NAV_serie2)
NAV_serie2<- sum(NAV_serie2)/24500000 #Shares Outstanding NAV final
######### Rendimiento serie 2 (Abril17- Feb18)###############
rend2<-(NAV_serie12/NAV_serie2)-1

In [43]:
rend2

[1] 0.2642

In [45]:
######### SERIE 3 (mayo 17) ###############
A3 <- as.character(base_datos[[3]][,1]) #mayo 17
B3 <- as.character(base_datos[[12]][,1]) #febrero 18

reg3 <- venta_compra(a=A3, b=B3)
shares_venta <- (base_datos[[3]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 3 ######
for(i in 1:length(reg3$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[3]]$tk == reg3$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta3 <- (base_datos[[3]][['shares']])
    titulos_venta3<- titulos_venta3[-139] 
    titulos_venta3<-replaceCommas(titulos_venta3)
    titulos_venta[k]<- (titulos_venta3[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[3]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta3<- vendi*Comision
    
  }
}
capital_dventa3<- (capital_dcompra2+ vendi)-gasto_venta3

In [46]:
gasto_venta3

[1] 17486

In [48]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 3######
for(j in 1:length(reg3$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg3$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra3 <- (base_datos[[12]][['shares']])
    titulos_compra3<- titulos_compra3[-143] 
    titulos_compra3<-replaceCommas(titulos_compra3)
    titulos_compra[p]<- (titulos_compra3[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra3<- compre*Comision
    
  }
}
capital_dcompra3<- (capital_dventa3 - compre)-gasto_compra3 #capital despues de comisiones por compra-venta


In [49]:
gasto_compra3

[1] 15997

In [50]:

######### NAV para rendimiento SERIE 3 (Mayo 17)  ###############
NAV_serie3 <- (base_datos[[3]][['mv']])
NAV_serie3<- NAV_serie3[-139] 
NAV_serie3<-replaceCommas(NAV_serie3)
NAV_serie3<- sum(NAV_serie3)/24950000 #Shares Outstanding NAV final
######### Rendimiento serie 3 (Mayo17-Feb18)###############
rend3<-(NAV_serie12/NAV_serie3)-1


In [51]:
rend3

[1] 0.2148

In [53]:
######### SERIE 4 (Junio 17) ###############
A4 <- as.character(base_datos[[4]][,1]) #junio 17
B4 <- as.character(base_datos[[12]][,1]) #febrero 18

reg4 <- venta_compra(a=A4, b=B4)
shares_venta <- (base_datos[[4]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 4 ######
for(i in 1:length(reg4$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[4]]$tk == reg4$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta4 <- (base_datos[[4]][['shares']])
    titulos_venta4<- titulos_venta4[-139] 
    titulos_venta4<-replaceCommas(titulos_venta4)
    titulos_venta[k]<- (titulos_venta4[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[4]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta4<- vendi*Comision
    
  }
}
capital_dventa4<- (capital_dcompra3+ vendi)-gasto_venta4

In [54]:
gasto_venta4

[1] 6847

In [56]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 3######
for(j in 1:length(reg4$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg4$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
   # print(p)
    titulos_compra4 <- (base_datos[[12]][['shares']])
    titulos_compra4<- titulos_compra4[-143] 
    titulos_compra4<-replaceCommas(titulos_compra4)
    titulos_compra[p]<- (titulos_compra4[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra4<- compre*Comision
    
  }
}
capital_dcompra4<- (capital_dventa4 - compre)-gasto_compra4 #capital despues de comisiones por compra-venta


In [57]:
 gasto_compra4

[1] 15545

In [58]:

######### NAV para rendimiento SERIE 4 (Junio 17)  ###############
NAV_serie4 <- (base_datos[[4]][['mv']])
NAV_serie4<- NAV_serie4[-139] 
NAV_serie4<-replaceCommas(NAV_serie4)
NAV_serie4<- sum(NAV_serie4)/24850000 #Shares Outstanding NAV final
######### Rendimiento serie 4 (Junio17-Feb18)###############
rend4<-(NAV_serie12/NAV_serie4)-1




In [59]:
rend4


[1] 0.2582

In [61]:
######### SERIE 5 (Julio 17) ###############
A5 <- as.character(base_datos[[5]][,1]) #junio 17
B5 <- as.character(base_datos[[12]][,1]) #febrero 18

reg5 <- venta_compra(a=A5, b=B5)
shares_venta <- (base_datos[[5]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 5 ######
for(i in 1:length(reg5$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[5]]$tk == reg5$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta5 <- (base_datos[[5]][['shares']])
    titulos_venta5<- titulos_venta5[-139] 
    titulos_venta5<-replaceCommas(titulos_venta5)
    titulos_venta[k]<- (titulos_venta5[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[5]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta5<- vendi*Comision
    
  }
}
capital_dventa5<- (capital_dcompra4+ vendi)-gasto_venta5


In [63]:
##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 5######
for(j in 1:length(reg5$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg5$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra5 <- (base_datos[[12]][['shares']])
    titulos_compra5<- titulos_compra5[-143] 
    titulos_compra5<-replaceCommas(titulos_compra5)
    titulos_compra[p]<- (titulos_compra5[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra5<- compre*Comision
    
  }
}
capital_dcompra5<- (capital_dventa5 - compre)-gasto_compra5 #capital despues de comisiones por compra-venta


In [64]:
######### NAV para rendimiento SERIE 5 (Julio 17)  ###############
NAV_serie5 <- (base_datos[[5]][['mv']])
NAV_serie5<- NAV_serie5[-139] 
NAV_serie5<-replaceCommas(NAV_serie5)
NAV_serie5<- sum(NAV_serie5)/25000000 #Shares Outstanding NAV final
######### Rendimiento serie original (mar17-feb18)###############
rend5<-(NAV_serie12/NAV_serie5)-1

In [65]:
rend5

[1] 0.2073

In [67]:
######### SERIE 6 (Agosto 17) ###############
A6 <- as.character(base_datos[[6]][,1]) #julioo 17
B6 <- as.character(base_datos[[12]][,1]) #febrero 18

reg6 <- venta_compra(a=A6, b=B6)
shares_venta <- (base_datos[[6]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 6 ######
for(i in 1:length(reg6$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[6]]$tk == reg6$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta6 <- (base_datos[[6]][['shares']])
    titulos_venta6<- titulos_venta6[-139] 
    titulos_venta6<-replaceCommas(titulos_venta6)
    titulos_venta[k]<- (titulos_venta6[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[6]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta6<- vendi*Comision
    
  }
}
capital_dventa6<- (capital_dcompra5+ vendi)-gasto_venta6

In [69]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 6 ######
for(j in 1:length(reg6$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg6$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra6 <- (base_datos[[12]][['shares']])
    titulos_compra6<- titulos_compra6[-143] 
    titulos_compra6<-replaceCommas(titulos_compra6)
    titulos_compra[p]<- (titulos_compra6[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra6<- compre*Comision
    
  }
}
capital_dcompra6<- (capital_dventa6 - compre)-gasto_compra6 #capital despues de comisiones por compra-venta


In [70]:

######### NAV para rendimiento SERIE 6 (Agosto 17)  ###############
NAV_serie6 <- (base_datos[[6]][['mv']])
NAV_serie6<- NAV_serie6[-139] 
NAV_serie6<-replaceCommas(NAV_serie6)
NAV_serie6<- sum(NAV_serie6)/24700000 #Shares Outstanding NAV final
######### Rendimiento serie original (agosto17-Feb18)###############
rend6<-(NAV_serie12/NAV_serie6)-1


In [71]:
rend6

[1] 0.1694

In [73]:
######### SERIE 7 (Septiembre 17) ###############
A7 <- as.character(base_datos[[7]][,1]) #julioo 17
B7 <- as.character(base_datos[[12]][,1]) #febrero 18

reg7 <- venta_compra(a=A7, b=B7)
shares_venta <- (base_datos[[7]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 7 ######
for(i in 1:length(reg7$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[7]]$tk == reg7$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta7 <- (base_datos[[7]][['shares']])
    titulos_venta7<- titulos_venta7[-144] 
    titulos_venta7<-replaceCommas(titulos_venta7)
    titulos_venta[k]<- (titulos_venta7[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[7]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta7<- vendi*Comision
    
  }
}
capital_dventa7<- (capital_dcompra6+ vendi)-gasto_venta7

In [74]:
gasto_venta7

[1] 7559

In [76]:
##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 7 ######
for(j in 1:length(reg7$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg7$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra7 <- (base_datos[[12]][['shares']])
    titulos_compra7<- titulos_compra7[-143] 
    titulos_compra7<-replaceCommas(titulos_compra7)
    titulos_compra[p]<- (titulos_compra6[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra7<- compre*Comision
    
  }
}
capital_dcompra7<- (capital_dventa7 - compre)-gasto_compra7 #capital despues de comisiones por compra-venta


In [77]:
gasto_compra7

[1] 9539

In [78]:
######### NAV para rendimiento SERIE 7 (Septiembre 17)  ###############
NAV_serie7 <- (base_datos[[7]][['mv']])
NAV_serie7<- NAV_serie7[-144] 
NAV_serie7<-replaceCommas(NAV_serie7)
NAV_serie7<- sum(NAV_serie7)/24600000 #Shares Outstanding NAV final
######### Rendimiento serie 7 (Septiembre17-Feb18)###############
rend7<-(NAV_serie12/NAV_serie7)-1


In [79]:
rend7

[1] 0.1677

In [81]:
######### SERIE 8 (Octubre 17) ###############
A8 <- as.character(base_datos[[8]][,1]) #Octubre 17
B8 <- as.character(base_datos[[12]][,1]) #febrero 18

reg8 <- venta_compra(a=A8, b=B8)
shares_venta <- (base_datos[[8]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 8 ######
for(i in 1:length(reg8$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[8]]$tk == reg8$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta8 <- (base_datos[[8]][['shares']])
    titulos_venta8<- titulos_venta8[-145] 
    titulos_venta8<-replaceCommas(titulos_venta8)
    titulos_venta[k]<- (titulos_venta8[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[8]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta8<- vendi*Comision
    
  }
}
capital_dventa8<- (capital_dcompra7+ vendi)-gasto_venta8

In [82]:
gasto_venta8

[1] 7944

In [84]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 8 ######
for(j in 1:length(reg8$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg8$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra8 <- (base_datos[[12]][['shares']])
    titulos_compra8<- titulos_compra8[-143] 
    titulos_compra8<-replaceCommas(titulos_compra8)
    titulos_compra[p]<- (titulos_compra8[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra8<- compre*Comision
    
  }
}



In [85]:
gasto_compra8

[1] 5571

In [86]:
capital_dcompra8<- (capital_dventa8 - compre)-gasto_compra8 #capital despues de comisiones por compra-venta

######### NAV para rendimiento SERIE 8 (octubre 17)  ###############
NAV_serie8 <- (base_datos[[8]][['mv']])
NAV_serie8<- NAV_serie8[-145] 
NAV_serie8<-replaceCommas(NAV_serie8)
NAV_serie8<- sum(NAV_serie8)/24450000 #Shares Outstanding NAV final
######### Rendimiento serie 8 (oct17-Feb18)###############
rend8<-(NAV_serie12/NAV_serie8)-1

In [87]:
rend8

[1] 0.08043

In [89]:
######### SERIE 9 (Noviembre 17) ###############
A9 <- as.character(base_datos[[9]][,1]) #novimbre 17
B9 <- as.character(base_datos[[12]][,1]) #febrero 18

reg9 <- venta_compra(a=A9, b=B9)
shares_venta <- (base_datos[[9]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 9 ######
for(i in 1:length(reg9$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[9]]$tk == reg9$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta9 <- (base_datos[[9]][['shares']])
    titulos_venta9<- titulos_venta9[-144] 
    titulos_venta9<-replaceCommas(titulos_venta9)
    titulos_venta[k]<- (titulos_venta9[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[9]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta9<- vendi*Comision
    
  }
}
capital_dventa9<- (capital_dcompra8+ vendi)-gasto_venta9

In [90]:
gasto_venta9

[1] 7082

In [92]:

##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 9 ######
for(j in 1:length(reg9$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg9$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra9 <- (base_datos[[12]][['shares']])
    titulos_compra9<- titulos_compra9[-143] 
    titulos_compra9<-replaceCommas(titulos_compra9)
    titulos_compra[p]<- (titulos_compra9[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra9<- compre*Comision
    
  }
}


In [93]:
gasto_compra9

[1] 5571

In [94]:
capital_dcompra9<- (capital_dventa9 - compre)-gasto_compra9 #capital despues de comisiones por compra-venta
capital_dcompra9

[1] 938032798

In [95]:

######### NAV para rendimiento SERIE 9 (Noviembre 17)  ###############
NAV_serie9 <- (base_datos[[9]][['mv']])
NAV_serie9<- NAV_serie9[-144] 
NAV_serie9<-replaceCommas(NAV_serie9)
NAV_serie9<- sum(NAV_serie9)/25100000 #Shares Outstanding NAV final
######### Rendimiento serie 9 (nov17-Feb18)###############
rend9<-(NAV_serie12/NAV_serie9)-1

In [96]:
rend9

[1] 0.07151

In [98]:
######### SERIE 10 (Diciembre 17) ###############
A10 <- as.character(base_datos[[10]][,1]) #diciembre 17
B10 <- as.character(base_datos[[12]][,1]) #febrero 18

reg10 <- venta_compra(a=A10, b=B10)
shares_venta <- (base_datos[[10]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 10 ######
for(i in 1:length(reg10$sequitaron)){ #ver posición de los tk que se quitaron en base de datos
  #venta<-c()
  venta[i]<- which(base_datos[[10]]$tk == reg10$sequitaron[i])
  for(k in 1:length(venta)){ # esas posiciones multiplicar precios por shares
    #titulos_venta<-c()
    #print(k)
    titulos_venta10 <- (base_datos[[10]][['shares']])
    titulos_venta10<- titulos_venta10[-143] 
    titulos_venta10<-replaceCommas(titulos_venta10)
    titulos_venta[k]<- (titulos_venta10[venta[k]])
    #dinero<-c()
    dinero[k]<- as.numeric(as.character(base_datos[[10]]$precios[venta[k]]))
    #dinero1<- as.numeric(as.character(dinero))
    vendi<-sum(dinero*titulos_venta)
    gasto_venta10<- vendi*Comision
    
  }
}
capital_dventa10<- (capital_dcompra9+ vendi)-gasto_venta10

In [99]:
gasto_venta10

[1] 2822

In [101]:
##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 10 ######
for(j in 1:length(reg10$seincluyeron)){ #ver posición de los tk que se incluyeron en base de datos
  #compra<-c()
  compra[j]<- which(base_datos[[12]]$tk == reg10$seincluyeron[j])
  for(p in 1:length(compra)){ # esas posiciones multiplicar precios por shares
    #titulos_compra<-c()
    #print(p)
    titulos_compra10 <- (base_datos[[12]][['shares']])
    titulos_compra10<- titulos_compra10[-143] 
    titulos_compra10<-replaceCommas(titulos_compra10)
    titulos_compra[p]<- (titulos_compra10[compra[p]])
    #dinero<-c()
    dinero[p]<- as.numeric(as.character(base_datos[[12]]$precios[compra[p]]))
    #dinero1<- as.numeric(as.character(dinero))
    compre<-sum(dinero*titulos_compra)
    gasto_compra10<- compre*Comision
    
  }
}
capital_dcompra10<- (capital_dventa10 - compre)-gasto_compra10 #capital despues de comisiones por compra-venta


In [102]:
capital_dcompra10

[1] 928859361

In [103]:
gasto_compra10

[1] 5571

In [104]:
######### NAV para rendimiento SERIE 10 (Dicimbre 17)  ###############
NAV_serie10 <- (base_datos[[10]][['mv']])
NAV_serie10<- NAV_serie10[-143] 
NAV_serie10<-replaceCommas(NAV_serie10)
NAV_serie10<- sum(NAV_serie10)/24950000 #Shares Outstanding NAV final
######### Rendimiento serie 10 (diciembre17-Feb18)###############
rend10<-(NAV_serie12/NAV_serie10)-1

In [105]:
rend10

[1] 0.09157

In [106]:


######### SERIE 11 (Enero 18) ###############
A11 <- as.character(base_datos[[11]][,1]) #enero 18
B11 <- as.character(base_datos[[12]][,1]) #febrero 18

reg11 <- venta_compra(a=A11, b=B11)
shares_venta <- (base_datos[[11]][['shares']])
##### CACULAR EL GASTO POR VENTAS DE TK SERIE 11 ######
#NO HUBO VENTAS
gasto_venta11 <-0
capital_dventa11<-capital_dcompra10
##### CACULAR EL GASTO POR COMPRAS DE TK SERIE 11 ######
#NO HUBO VENTAS
gasto_compra11 <-0
capital_dcompra11<- capital_dventa11 #capital despues de comisiones por compra-venta

######### NAV para rendimiento SERIE 11 (Enero 18)  ###############
NAV_serie11 <- (base_datos[[11]][['mv']])
NAV_serie11<- NAV_serie11[-143] 
NAV_serie11<-replaceCommas(NAV_serie11)
NAV_serie11<- sum(NAV_serie11)/24300000 #Shares Outstanding NAV final
######### Rendimiento serie 11 (ene18 - Feb18)###############
rend11<-(NAV_serie12/NAV_serie11)-1



In [107]:
rend11

[1] 0.005266

#### DATAFRAME FINAL (1)

In [108]:
######### DATAFRAME FINAL ##########

rend_series <- c(rend1*100, rend2*100, rend3*100, rend4*100, rend5*100, rend6*100, rend7*100, rend8*100, rend9*100, rend10*100, rend11*100)
comision1<-0
comision_series <- c(comision1, gasto_venta2 + gasto_compra2, gasto_venta3 + gasto_compra3,
                     gasto_venta4 + gasto_compra4,gasto_venta5 + gasto_compra5, gasto_venta6 + gasto_compra6, 
                     gasto_venta7 + gasto_compra7, gasto_venta8 + gasto_compra8,gasto_venta9 + gasto_compra9,
                     gasto_venta10 + gasto_compra10, gasto_venta11 + gasto_compra11)
capital_final <- c(Capital_Inicial,capital_dcompra2, capital_dcompra3, capital_dcompra4, capital_dcompra5,capital_dcompra6,
                  capital_dcompra7, capital_dcompra8, capital_dcompra9, capital_dcompra10, capital_dcompra11)
dataframe_final <- data.frame(matrix(ncol=4, nrow=11, data=0))

In [109]:
colnames(dataframe_final) <- c("Series", "Rends (%)", "Comisiones ($)", "Capital final ($)")

for(i in 1:11) {
  dataframe_final$Series[i] <- paste("Serie_", i, sep="")
}

{
  dataframe_final$`Rends (%)` <- rend_series
}

{
  dataframe_final$`Comisiones ($)` <- comision_series
}

{
  dataframe_final$`Capital final ($)` <- capital_final
}


___
#### <Strong> <font color = skyblue> TABLA 1. Análisis de rendimiento y comisiones por rebalanceo </Strong>

In [110]:
dataframe_final

Series,Rends (%),Comisiones ($),Capital final ($)
Serie_1,28.6284,0,999000000
Serie_2,26.4223,32711,1001356397
Serie_3,21.4792,33483,1006284066
Serie_4,25.8210,22392,977267810
Serie_5,20.7314,25498,958600276
Serie_6,16.9392,23037,931732602
Serie_7,16.7680,17098,925114436
Serie_8,8.0433,13515,933009311
Serie_9,7.1513,12653,938032798
Serie_10,9.1574,8393,928859361


> <p style='text-align: justify;'>En el DataFrame anterior se muestra los rendimientos de las 11 series, cabe mencionar que en la primer serie se considero que no habria rebalanceo, es decir las acciones que habia en marzo 17 se dejaron para marzo 18. Lo anterior con el objetivo de visualizar cual seria el rendimiento sin hacer ninguna compra o venta de acciones. Por esta misma razón existe un monto de $0.00 usd en comisión y el capital final es el mismo capital inicial de 999000000 usd que se propuso al inicio.

> <p style='text-align: justify;'>De la serie 2 hasta la ultima, se hicieron las compras y ventas correspondientes.En la columna de Comisiones se puede observar dependiendo la serie en cuanto dinero incurre el inversionista por que rebalanceen su portafolio, se puede notar que en la serie 3 es donde se gasta mas en comisiones con un total de $33,483 dls. Y en la ultima columna se aprecia como disminuye el capital conforme se acumulan las comisiones.

> <p style='text-align: justify;'>En general en torno al rendimiento se concluye para este ETF que el haber rebalanceado no hizo que mejorara su rendimiento ya que el rendimiento mayor es cuando las acciones se quedaron estaticas. los rendimientos son decrecientes ya que entre mayor es el plazo se puede obtener un mejor retorno en contraste con tener una inversión a ccorto plazo como lo fue el caso de la serie 11 donde el intervalo de tiempo due de enero 18 a febrero 18 tniendo una rentabilidad del .52%

___
#### <Strong> <font color = skyblue> GRAFICAS </Strong>

> <p style='text-align: justify;'>Cada código de su respectiva grafica te abre una pestaña nueva en internet que te dirige hacia mi perfil de PLOTLY en el cual tengo guardados todos los gráficos. Las gráficas son iteractivas por lo que desde mi perfil podras manejarlas mejor. 

> <p style='text-align: justify;'> NOTA*** Al final anexo los links de las graficas de mi perfil de plotly en caso de no querer correr todo el notebook.

In [113]:
######### Gráfica 1 de comisiones ##########


Sys.setenv("plotly_username"="jazerape")
Sys.setenv("plotly_api_key"="eX7PWHChSnlXoCQOaddE")


Series <- c('Serie 1','Serie 2','Serie 3','Serie 4','Serie 5','Serie 6','Serie 7','Serie 8','Serie 9','Serie 10','Serie 11')
data <- data.frame(Series, comision_series)
p <- plot_ly(data, x = Series,y = comision_series, type = 'scatter', mode = 'lines', color = I('pink')) %>%
  layout(title = 'Monto($) por comisiones de rebalanceo por c/serie del U.S. Technolgy ETF',
         xaxis = list(title = 'No. de Serie'),
         yaxis = list (title = 'Comisiones (USD)'))

# Create a shareable link to your chart
# Set up API credentials: https://plot.ly/r/getting-started
chart_link = api_create(p, filename="Grafica 1")
chart_link  #Te manda directo al link de mi perfil donde guarde los graficos, la grafica es iterativa

Found a grid already named: 'Grafica 1 Grid'. Since fileopt='overwrite', I'll try to update it
Found a plot already named: 'Grafica 1'. Since fileopt='overwrite', I'll try to update it


> <p style='text-align: justify;'>En el primer gáfico se observa el Monto($) por comisiones de rebalanceo por c/serie del U.S. Technolgy ETF, es facil notar que en la serie 2 fue en la que se incurrio en mas gasto en la compra-venta de acciones. en general se muestra una tendencia decreciente, conforme el intervalo de tiempo disminuye, es decir entre la serie de tiempo se acerca a la onceava los movimientos son menos.

Link del primer gráfico
https://plot.ly/~jazerape/8/#plot


In [114]:
##### Grafica 2 NAV de todo el ETF con rebalanceo#####
Fechag2 <- c('Mar','Abr','May','Jun','Jul','Ago','Sept','Oct','Nov','Dic', 'Ene')
NAV <- c(NAV_serie1,NAV_serie2,NAV_serie3,NAV_serie4,NAV_serie5,NAV_serie6,NAV_serie7,NAV_serie8,NAV_serie9,NAV_serie10,NAV_serie11)
data2 <- data.frame(Fechag2, NAV)
p <- plot_ly(data2, x = Fechag2,y = NAV, type = 'scatter', mode = 'lines', color = I('green')) %>%
  layout(title = 'Net Asset Value del U.S. Technolgy ETF',
         xaxis = list(title = 'Fechas'),
         yaxis = list (title = 'Valor (USD)'))

# Create a shareable link to your chart
# Set up API credentials: https://plot.ly/r/getting-started
chart_link = api_create(p, filename="Grafica 2")
chart_link #Te manda directo al link de mi perfil donde guarde los graficos, la grafica es iterativa

Found a grid already named: 'Grafica 2 Grid'. Since fileopt='overwrite', I'll try to update it
Found a plot already named: 'Grafica 2'. Since fileopt='overwrite', I'll try to update it


> <p style='text-align: justify;'>El Net Asset Value nos indica cuánto realmente vale el ETF, considerando todos los instrumentos que tiene en la “panza” este instrumento de inversión. Para el U.S.Technology se aprecia una tedencia creciente con unos ligeros picos en donde el valor tuvo pequeñas caidas y subidas. Estos pequeños picos es debido a que el Precio de Mercado de cada uno de los instrumentos que el ETF tiene en la “panza” cambia, por lo que cada cambio puede incrementar o disminuir el valor de dicho ETF. Hay que tomar en cuenta que al igual que el precio, el NAV está en constante cambio durante la sesión bursátil.

Link del segundo gráfico
https://plot.ly/~jazerape/10/#/

In [115]:
######## Grafica 3 Comparativa valor portafolio Y COMISIONES ########
Val_port<-c((sum(NAV_serie1))*24300000, (sum(NAV_serie2))*24500000,(sum(NAV_serie3))*24950000,(sum(NAV_serie4))*24850000,sum(NAV_serie5)*25000000,(sum(NAV_serie6))*24700000,(sum(NAV_serie7))*24600000,(sum(NAV_serie8))*24450000,(sum(NAV_serie9))*25100000,(sum(NAV_serie10))*24950000,(sum(NAV_serie11))*24300000)
comision_acum <- c(comision_series[1], comision_series[1]+comision_series[2],comision_series[1]+comision_series[2]+comision_series[3],comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4],comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5],
             comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6],comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6]+comision_series[7],comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6]+comision_series[7]+comision_series[8],
             comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6]+comision_series[7]+comision_series[8]+comision_series[9],comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6]+comision_series[7]+comision_series[8]+comision_series[9]+comision_series[10], 
             comision_series[1]+comision_series[2]+comision_series[3]+comision_series[4]+comision_series[5]+comision_series[6]+comision_series[7]+comision_series[8]+comision_series[9]+comision_series[10]++comision_series[11])
ay <- list(
  tickfont = list(color = "pink"),
  overlaying = "y",
  title = "Monto en usd"
)
p <- plot_ly() %>%
  add_lines(x = Fechag2, y = Val_port, name = "Val. Portafolio",color = I('violet')) %>%
  add_lines(x = Fechag2, y = comision_acum, name = "Comisiones acumuladas", yaxis = "y2",color = I('turquoise')) %>%
  layout(
    title = "Comprativo Valor del Portafolio y Comisiones", yaxis2 = ay,
    xaxis = list(title="Fechas")
  )

# Create a shareable link to your chart
# Set up API credentials: https://plot.ly/r/getting-started
chart_link = api_create(p, filename="Gráfica 3")
chart_link


Found a grid already named: 'Gráfica 3 Grid'. Since fileopt='overwrite', I'll try to update it
Found a plot already named: 'Gráfica 3'. Since fileopt='overwrite', I'll try to update it


> <p style='text-align: justify;'>Por último tnemos este gráfico comprativo entre el Valor del Portafolio y las Comisiones. 

Link del tercer gráfico
https://plot.ly/~jazerape/23/#/

___
### <Strong> <font color = skyblue> Métricas de Desempeño </Strong>

Segunda parte: Se plantea obtener el riesgo en el que se esta incurriendo para cada serie cada series (desviación estandar), el sharpe ratio y por ultimo Traynor.

#### Riesgo
> <p style='text-align: justify;'>Dada una determinada rentabilidad en una inversión, cuanto mayor sea la desviación estándar, mayor será el riesgo. Con la desviación estándar podemos cuantificar cuál será el intervalo en el cual caerá una determinada rentabilidad futura en una inversión o también cuál será la probabilidad de obtener un rendimiento esperado.


#### Sharpe Ratio
> <p style='text-align: justify;'>Este índice mide la prima de riesgo del fondo por unidad de riesgo (volatilidad), representa la pendiente de la recta que une el rendimiento del activo libre de riesgo y el fondo. Su resultado permite determinar cuáles fondos han logrado batir al índice de referencia o benchmark y cuáles han sido superados por este, es entonces, una medida de desempeño que permite cuestionar la capacidad del gestor en caso tal que su resultado sea inferior al de una cartera no gestionada (que replique el índice) o al de cualquier combinación entre el activo libre de riesgo y la canasta del mercado.


#### El índice de Traynor
> <p style='text-align: justify;'>Es medido como la prima de riesgo del fondo por unidad de riesgo beta de este, lo cual se traduce en la pendiente de la recta que une el rendimiento del activo libre de riesgo con el rendimiento del fondo que se está evaluando.Su resultado permite ordenar o jerarquizar el conjunto de fondos medidos y también comparar su desempeño contra el mercado (teniendo en cuenta que este índice parte de la Línea del Mercado de Valores - SLM). Así se puede observar cuáles portafolios batieron al benchmark y cuáles no, de esta manera un buen gestor sería aquel que seleccionó activos que le proporcionaron una prima media por unidad de riesgo superior a la del mercado, de lo contrario un inversionista habría obtenido mejores resultados invirtiendo simplemente en el índice del mercado. 

> <p style='text-align: justify;'>Este ratio presupone que la Beta es un buen indicador de riesgo sistemático o de mercado, asume la verificación del modelo CAPM lo que lo limita frente a otros índices como el de Sharpe.



In [116]:
suppressMessages(library(openxlsx)) # Tablas en HTML
suppressMessages(library(readxl))

In [117]:
#### Archivo con historicos del NAV del ETF
iShares_US_Technology_ETF_fund <- read_excel("C:/Users/User/Documents/Trading/lab2/iShares-US-Technology-ETF_fund.xlsx")
#read_excel("C:/Users/user/Documents/Work/Data.xlsx")

Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Expecting numeric in B1750 / R1750C2: got '--'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Expecting numeric in B1751 / R1751C2: got '--'"

In [118]:
iShares_US_Technology_ETF_fund

As Of,Index Level,NAV per Share,Ex-Dividends,Shares Outstanding
"Mar 08, 2019",2343,181.8,--,21550000
"Mar 07, 2019",2342,181.8,--,21600000
"Mar 06, 2019",2367,183.7,--,21500000
"Mar 05, 2019",2379,184.7,--,21500000
"Mar 04, 2019",2378,184.6,--,21550000
"Mar 01, 2019",2381,184.8,--,21550000
"Feb 28, 2019",2363,183.4,--,21550000
"Feb 27, 2019",2373,184.2,--,21550000
"Feb 26, 2019",2375,184.3,--,21550000
"Feb 25, 2019",2372,184.1,--,21600000


In [119]:
##### Calculo de desviaciones ######
historico1 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:488]
desvest1 <- sd(historico1)

historico2 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:469]
desvest2 <- sd(historico2)

historico3 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:447]
desvest3 <- sd(historico3)

historico4 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:425]
desvest4 <- sd(historico4)

historico5 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:405]
desvest5 <- sd(historico5)

historico6 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:382]
desvest6 <- sd(historico6)

historico7 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:362]
desvest7 <- sd(historico7)

historico8 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:340]
desvest8 <- sd(historico8)

historico9 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:319]
desvest9 <- sd(historico9)


historico10 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:299]
desvest10 <- sd(historico10)

 
historico11 <- iShares_US_Technology_ETF_fund$`NAV per Share`[259:278]
desvest11 <- sd(historico11)

In [126]:
############# sharpe de las series ##################################
usd_rate = c(0.0241,0.0241,0.0239,0.0240,0.0241,0.0241,0.0243,0.0243,0.0242,0.0242,0.0241) #tasa libre de riesgo EEUU #0.0241
sharpe1 = (rend1-usd_rate[1])/desvest1
sharpe2 = (rend2-usd_rate[2])/desvest2
sharpe3 = (rend3-usd_rate[3])/desvest3
sharpe4 = (rend4-usd_rate[4])/desvest4
sharpe5 = (rend5-usd_rate[5])/desvest5
sharpe6 = (rend6-usd_rate[6])/desvest6
sharpe7 = (rend7-usd_rate[7])/desvest7
sharpe8 = (rend8-usd_rate[8])/desvest8
sharpe9 = (rend9-usd_rate[9])/desvest9
sharpe10 = (rend10-usd_rate[10])/desvest10
sharpe11 = (rend11-usd_rate[11])/desvest11

In [127]:
###################### dATAfRAME final (2) ##########################
desviaciones <- c(desvest1, desvest2, desvest3, desvest4, desvest5, desvest6, desvest7,
                  desvest8, desvest9, desvest10, desvest11)

sharpe_series <- c(sharpe1,sharpe2,sharpe3,sharpe4,sharpe5,sharpe6,sharpe7,sharpe8,sharpe9,sharpe10,sharpe11)



In [155]:
historicos <- c(NAV_serie1,NAV_serie2,NAV_serie3,NAV_serie4,NAV_serie5,NAV_serie6,NAV_serie7,NAV_serie8,NAV_serie9,NAV_serie10,NAV_serie11)
beta_etf<- cov(historicos, usd_rate)
tasas_desvest = sd(usd_rate)
historicos_desvest = sd(historicos)
beta_port = ((historicos_desvest/tasas_desvest)*beta_etf) #[1][1]
beta_port

[1] 70.27

In [156]:
#TREYNOR RATIO DE LAS SERIES
rendd_series<- c(rend1,rend2,rend3,rend4,rend5,rend6,rend7,rend8,rend9,rend10,rend11)
treynor_series = (((rendd_series)-usd_rate)/beta_port)*100 #/100


In [157]:
dataframe_final <- data.frame(matrix(ncol=5, nrow=11, data=0))
colnames(dataframe_final) <- c("Series","Rendimiento (%)", "Riesgo", "Sharpe", "Traynor")

for(i in 1:11) {
  dataframe_final$Series[i] <- paste("Serie_", i, sep="")
}

{
  dataframe_final$`Rendimiento (%)` <- rend_series
}

{
  dataframe_final$Riesgo <- desviaciones
}


{
  dataframe_final$Sharpe <- sharpe_series
}

{
  dataframe_final$Traynor <- treynor_series
}

___
#### <Strong> <font color = skyblue> TABLA 2. Análisis de las métricas de desempeño por serie </Strong>

In [158]:
dataframe_final

Series,Rendimiento (%),Riesgo,Sharpe,Traynor
Serie_1,28.6284,11.922,0.021992,0.37309
Serie_2,26.4223,11.090,0.021652,0.34170
Serie_3,21.4792,10.738,0.017777,0.27164
Serie_4,25.8210,10.414,0.022491,0.33328
Serie_5,20.7314,9.692,0.018904,0.26072
Serie_6,16.9392,8.549,0.016996,0.20675
Serie_7,16.7680,7.088,0.020228,0.20403
Serie_8,8.0433,4.956,0.011326,0.07988
Serie_9,7.1513,5.210,0.009081,0.06733
Serie_10,9.1574,4.640,0.014520,0.09587


> <p style='text-align: justify;'>En la tabla 2. se puede concluir que fue mejor no hacer rebalanceo para el intervalo de tiempo marzo17 - marzo 18 ya que el rendimiento fue un poco mas alto que cuando se rebalanceo sin embargo el riesgo es mas alto por lo que depende del perfil del inversionar decidir no hacer rebalanceo y solo replicar el índice. Tanto el rendimiento como el riesgo tienen una tendencia decreciente cada vez que el intervalo de tiempo es mas pequeño. 

> <p style='text-align: justify;'>Hay cuatro puntos importantes para anañizar el Sharpe Ratio:
* Cuanto mayor es el Sharpe Ratio, mejor es la rentabilidad del fondo en relación a la cantidad de riesgo que se ha tomado en la inversión.
* Si el ratio de Sharpe es negativo, indica un rendimiento inferior al de la rentabilidad sin riesgo.
* Todo ratio de Sharpe inferior a 1 supone que el rendimiento del activo es inferior al riesgo que estamos asumiendo al invertir en el mismo.
* Cuando la volatilidad del fondo es grande, asumimos más riesgo; mayor es el denominador de la ecuación y menor es el Sharpe Ratio, a no ser que el rendimiento del fondo consiga compensar esa mayor volatilidad.

> <p style='text-align: justify;'>Tomando en cuenta lo anterior, se puede inferir que en todas las series el rendimiento es inferior al riesgo que se esta asumiendo al invertir en ese tramo. Por otro lado la serie 11 muestra un sharpe ratio negativo por lo que su rendimiento es infrior al de la rentabilidad sin riesgo por lo que no conviene invertir en dicha serie, sería mejor si se invierte en deuda pública.

> <p style='text-align: justify;'>Para la tercera medida de desempeño cuanto mayor sea el ratio de Treynor, mayor será el rendimiento ponderado y más interesante resultará el producto para el inversor. Es decir, mejor gestión del fondo. Para est laboratorio el que obtuvo mejor traynor fue la serie 1.


In [161]:
########## PARTE 2 Definir una regla con base a la cual se planea hacer rebalanceo